In [8]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import DistilBertForSequenceClassification, Trainer,TrainingArguments
import accelerate
import transformers
from transformers import DistilBertTokenizerFast
import numpy as np
import os
from datasets import load_dataset
import torch
from transformers import DistilBertModel, DistilBertTokenizer
import pandas as pd


#load data from csv files 

text = pd.read_csv('jigsaw_toxicity_comment/toxic_test.csv')
text_labels = pd.read_csv('jigsaw_toxicity_comment/test_labels.csv')

#merge the two dataframes on the id column
data = pd.merge(text, text_labels, on='id')

#drop rows with -1 in any of the columns

data = data[data.toxic != -1]
data = data[data.severe_toxic != -1]
data = data[data.obscene != -1]
data = data[data.threat != -1]
data = data[data.insult != -1]
data = data[data.identity_hate != -1]

data.reset_index(drop=True, inplace=True)
data

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0001ea8717f6de06,Thank you for understanding. I think very high...,0,0,0,0,0,0
1,000247e83dcc1211,:Dear god this site is horrible.,0,0,0,0,0,0
2,0002f87b16116a7f,"""::: Somebody will invariably try to add Relig...",0,0,0,0,0,0
3,0003e1cccfd5a40a,""" \n\n It says it right there that it IS a typ...",0,0,0,0,0,0
4,00059ace3e3e9a53,""" \n\n == Before adding a new product to the l...",0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
63973,fff8f64043129fa2,":Jerome, I see you never got around to this…! ...",0,0,0,0,0,0
63974,fff9d70fe0722906,==Lucky bastard== \n http://wikimediafoundatio...,0,0,0,0,0,0
63975,fffa8a11c4378854,==shame on you all!!!== \n\n You want to speak...,0,0,0,0,0,0
63976,fffac2a094c8e0e2,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0,1,0,1,0


In [12]:
#load model and tokenzer from pretrained local directory in folder jigsaw_toxicity_comment_model/

model = DistilBertForSequenceClassification.from_pretrained('jigsaw_toxicity_comment_model/')
tokenizer = DistilBertTokenizerFast.from_pretrained('jigsaw_toxicity_comment_model/')

#try to predict the 6 labels of the test daata using the model on only one sample, print the text, the predciton and the true label
sample = data['comment_text'][1000]
sample_label = data.iloc[1000,2:8].values

inputs = tokenizer(sample, return_tensors='pt')
outputs = model(**inputs)
predictions = torch.sigmoid(outputs.logits).detach().numpy().squeeze()
print(f"Text: {sample}")
print(f"Prediction: {predictions}")
print(f"True label: {sample_label}")



Text: " 
 == Speedy deletion of ""Mason Henson"" == 

  A page you created, Mason Henson, has been tagged for deletion, as it meets one or more of the criteria for speedy deletion; specifically, it is about a person or group of people, but it does not indicate how they are important or significant, and thus why they should be included in an encyclopedia. Please see the guidelines for what is generally accepted as notable, and the guidelines for biographies in particular. 

 You are welcome to contribute content which complies with our content policies and any applicable inclusion guidelines. However, please do not simply re-create the page with the same content. You may also wish to read our introduction to editing and guide to writing your first article. 

 Thank you. HG /  /  "
Prediction: [1.7631003e-04 2.3163520e-05 1.2691977e-04 8.8013141e-05 7.3324729e-05
 1.1384001e-04]
True label: [0 0 0 0 0 0]
